In [1]:
import base64
import ssl
import geopy.distance
import googlemaps
import pandas as pd
import requests
from datetime import datetime
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import math
import pylab
from shapely.geometry import Point
from pyproj import Transformer
from shapely.ops import transform
from bs4 import BeautifulSoup
from urllib.request import urlopen
from shodan import Shodan
from dateutil.relativedelta import relativedelta
import time
from flask import Flask, request,render_template, Response
from flask_restful import Resource, Api
from flask_cors import CORS, cross_origin
from gevent.pywsgi import WSGIServer
import os
import platform
import json
import unidecode
import urllib.parse
import mobility_funs as mf
import mobility_module as mm
import argparse
import youtube_module
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)

In [3]:
GOOGLE_API_KEY= ""
SHODAN_API_KEY=""
api_key_weather = ''

In [4]:
#It gathers all the points defining the circunference around (latitude,longitude) with radius equal to "error"
def get_points_circunference(latitude,longitude,error):
    local_azimuthal_projection = "+proj=aeqd +R=6371000 +units=m +lat_0={} +lon_0={}".format(latitude, longitude)
    wgs84_to_aeqd = Transformer.from_proj('+proj=longlat +datum=WGS84 +no_defs', local_azimuthal_projection)
    aeqd_to_wgs84 = Transformer.from_proj(local_azimuthal_projection, '+proj=longlat +datum=WGS84 +no_defs')
    point_transformed = Point(wgs84_to_aeqd.transform(latitude, longitude))
    buffer = point_transformed.buffer(error)
    circle = transform(aeqd_to_wgs84.transform, buffer)
    return list(circle.exterior.coords)

#It collects all the points each "distance" meters in the circunference around the coordinates
def get_all_points_around(latitude,longitude,error,distance):
    res=[]
    for i in range(distance,error+distance,distance):
        for p in get_points_circunference(latitude,longitude,i):
            res.append(p)
    return res

#It filters the points to keep only one each "distance" meters
def filter_points_distance(points,distance):
    res=[]
    last_point_added=(0,0)
    for p in points:
        dif=geopy.distance.geodesic(p, last_point_added).m
        if dif > distance:
            res.append(p)
            last_point_added=p
    return res

In [5]:
def generate_youtube_search(latitude,longitude): #Date format: 2020-08-11
    return "https://mattw.io/youtube-geofind/location?location="+str(latitude)+","+str(longitude)+"&radius=5&keywords=lora&doSearch=true"

def generate_twitter_search(latitude,longitude,date_init,date_end):
    if date_init=="":
        return "https://twitter.com/search?q=lora%20near%3A"+str(latitude)+"%2C"+str(longitude)+"&src=typed_query&f=live"
    else: 
        return "https://twitter.com/search?q=lora%20near%3A"+str(latitude)+"%2C"+str(longitude)+"%20until%3A"+str(date_end)+"%20since%3A"+str(date_init)+"&src=typed_query&f=live"


In [6]:
def get_cadastral_information(latitude,longitude):
    url="https://www1.sedecatastro.gob.es/CYCBienInmueble/OVCListaBienes.aspx?del=30&muni=28&rc1=0020006&rc2=00WH72F&from=OVCBusqueda&final=&pest=coordenadas&latitud="+str(latitude)+"&longitud="+str(longitude)+"&gradoslat=&minlat=&seglat=&gradoslon=&minlon=&seglon=&x=&y=&huso=0&tipoCoordenadas=2&TipUR=Coor"
    
    try:
        gcontext = ssl.SSLContext()
        page = urlopen(url, context=gcontext)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        bad_string = "  copiar       código de barras"

        tags = soup.findAll('label', {'class': lambda x: x and 'control-label black text-left' in x})

        res=[]
        for s in tags:
            res.append(s.get_text(separator=" ").strip().replace(bad_string, ""))
        ciudad=res[0]
        #res=[res[0],res[1]]
    except:
        res=["Not available","Not available"]
    return res

In [7]:
def get_maps_image(latitude,longitude):
    url="https://maps.googleapis.com/maps/api/staticmap?center=" + str(latitude) +",+" + str(longitude) +"&zoom=17&scale=1&size=600x300&maptype=satellite&format=png&visual_refresh=true&key=" + GOOGLE_API_KEY
    img = requests.get(url, stream=True)
    str_equivalent_image = base64.b64encode(img.content).decode()
    img_tag = "<img src='data:image/png;base64," + str_equivalent_image + "'/>"
    return img_tag

def get_maps_city(latitude,longitude):
    gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
    reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))
    return reverse_geocode_result[0]["address_components"][2]["long_name"]

def get_maps_province(latitude,longitude):
    gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
    google_address=gmaps.reverse_geocode((latitude,longitude))
    return google_address[0]["address_components"][3]["long_name"]

def get_maps_address(latitude,longitude):
    gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
    reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))
    return reverse_geocode_result[0]["formatted_address"]


def get_nearby_places(latitude,longitude,distance):
    if distance == 0:
        distance = 1000
    gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
    reverse_geocode_result = gmaps.places_nearby(location=(latitude, longitude), radius=distance)
    res=[]
    for r in reverse_geocode_result["results"]:
        res.append((r["name"]))
    return res

In [8]:
def shodan_search(term):
    api = Shodan(SHODAN_API_KEY)
    term=unidecode.unidecode(term)
    r = api.search(term)
    url = "https://www.shodan.io/search?query=" + str(urllib.parse.quote_plus(term))

    res = {}
    res["Unknown"] = []
    for i in r["matches"]:
        if ("product" in i.keys()):
            if i["product"] not in res.keys():
                res[i["product"]] = []
            res[i["product"]].append(i["ip_str"])
        else:
            res["Unknown"].append(i["ip_str"])
    return url,res;

In [9]:
def generate_shodan_html(url,res):
    html='<a href="'+url+'" target="_blank">Shodan</a><br>'
    #html+='<button type = "button" class ="collapsible" > + </button >'
    #html += '<div class ="content">'
    #for k in res.keys():
     #   html += '<button type = "button" class ="collapsible" >'+str(k + " " + str(len(res[k])))+'</button >'
      #  html += '<div class ="content">'
       # html += '<p>'+str(res[k])+'</p>'
        #html += '</div>'
    #html+='</div>\n'
    #html += '<script> var coll = document.getElementsByClassName("collapsible"); var i; for (i = 0; i < coll.length; ' \
     #       'i++) { coll[i].addEventListener("click", function() {this.classList.toggle("active"); var content = ' \
      #      'this.nextElementSibling; if (content.style.display === "block") {content.style.display = "none"; } else ' \
       #     '{content.style.display = "block";}});}</script> '
    return html

In [10]:
def get_ttn_gateway_data(gateway_id,start_date,end_date):
    return "https://ttnmapper.org/gateways/csv.html?gateway="+gateway_id+"&startdate="+str(start_date)+"&enddate="+str(end_date)+"&gateways=on&lines=on&points=on"

In [11]:
def get_gpsjam_link(latitude,longitude,date):
    return "https://gpsjam.org/?lat="+str(latitude)+"&lon="+str(longitude)+"&z=8.0&date="+str(date)

In [12]:
def generate_html_popup(latitude,longitude,error,date):
    text_popup="<b>"+str(latitude)+","+str(longitude)+"</b><br>"
    text_popup +="<br>"
    
    
    text_popup +="<b>Google Maps Address:</b><br>"
    google_address=get_maps_address(latitude,longitude)
    text_popup+=google_address+"<br>"
    text_popup+="<br>"
    
    text_popup+="<b>Nearby interesting places:</b><br>"
    places=get_nearby_places(latitude,longitude,error)
    num_suggestions=5
    for i in range(0,num_suggestions,1):
        if len(places)>i:
            text_popup += places[i] + "<br>"
    text_popup += "<br>"
    
    if "Spain" in google_address or "España" in google_address:
        text_popup +="<b>Cadastral information:</b><br>"
        cadastral_data=get_cadastral_information(latitude,longitude)
        for i in cadastral_data:
            text_popup+=i+"<br>"
        text_popup +="<br>"
    
    #print(places[0])
    #url_shodan,list_shodan=shodan_search(places[0])
    #text_popup+=generate_shodan_html(url_shodan,list_shodan)
    #print("Devices found in shodan:\n")
    #for k in list_shodan.keys():
     #   print(k+" "+str(len(list_shodan[k])))
      #  print(list_shodan[k])

    gpsjam_link=get_gpsjam_link(latitude,longitude,date)
    text_popup += '<a href="'+gpsjam_link + '" target="_blank">GPS Jamming</a><br>'

    next_day = (datetime.strptime(date, '%Y-%m-%d').date() + relativedelta(days=1)).strftime('%Y-%m-%d')

    #twitter_search=generate_twitter_search(latitude,longitude,date,next_day)
    #text_popup += '<a href="'+twitter_search + '" target="_blank">Tweets</a><br>'
    
    #youtube_videos=generate_youtube_search(latitude,longitude)
    #text_popup += '<a href="'+youtube_videos + '" target="_blank">Youtube Videos</a><br>'

    img_tag=get_maps_image(latitude,longitude)
    text_popup+=img_tag+"<br>"
    return text_popup

In [13]:
def generate_static_html_folium_marker(latitude,longitude,error,date):
    distance_between_circles=200
    distance_between_points=100
    
    location=(latitude,longitude)
    
    all_points=filter_points_distance(get_all_points_around(latitude,longitude,error,distance_between_circles),distance_between_points)
    df_external_circle = pd.DataFrame(get_points_circunference(latitude, longitude, error), columns=["lat", "lon"], index=None)
    
    m = folium.Map(location=df_external_circle[["lat", "lon"]].mean().to_list(), zoom_start=16)
    marker_cluster = MarkerCluster().add_to(m)

    list_locations=[]
    for i, r in df_external_circle.iterrows():
        location2 = (r["lat"], r["lon"])
        list_locations.append(location2)
    
    cent = (sum([p[0] for p in list_locations]) / len(list_locations), sum([p[1] for p in list_locations]) / len(list_locations))
    list_locations.sort(key=lambda p: math.atan2(p[1] - cent[1], p[0] - cent[0]))
    folium.Polygon(list_locations, fill_color="yellow", fill_opacity=0.3).add_to(m)
    
    text_popup=generate_html_popup(latitude,longitude,error,date)
    
    
    folium.Marker(location=location,popup=text_popup).add_to(m)  # display the map
        
    return m
    

In [14]:
def generate_dynamic_html_folium_marker(lat_start,lon_start,lat_end,lon_end,date):
    distance_between_circles=200
    distance_between_points=100
    
    location_start=(lat_start,lon_start)
    location_end=(lat_end,lon_end)
    
    #all_points=filter_points_distance(get_all_points_around(latitude,longitude,error,distance_between_circles),distance_between_points)
    #df_external_circle = pd.DataFrame(get_points_circunference(latitude, longitude, error), columns=["lat", "lon"], index=None)
    
    m = folium.Map(location=location_start, zoom_start=16)
    marker_cluster = MarkerCluster().add_to(m)
    
    text_popup_start=generate_html_popup(lat_start,lon_start,0,date)
    text_popup_end=generate_html_popup(lat_end,lon_end,0,date)

    folium.Marker(location=location_start,popup=text_popup_start).add_to(m)  # display the map
    folium.Marker(location=location_end,popup=text_popup_end, icon=folium.Icon(color='black')).add_to(m)  # display the map
    
    return m
    

In [15]:
def parse_youtube(latitude=None, longitude=None, date=None):
    api_key = ''
    
    if date is None:
        date = '2022-10-05'
    if latitude is None:
        latitude = str(38.263375)
    if longitude is None:
        longitude = str(-0.737125)

    date_next = str((pd.to_datetime(date) + pd.DateOffset(1)).date())
    
    response = requests.get(f'https://youtube.googleapis.com/youtube/v3/search?part=snippet&location={latitude}%2C{longitude}&locationRadius=1km&publishedAfter={date}T00%3A00%3A00Z&q=lora&type=video&key={api_key}')
    return response.json()

def process_yb_json(json_data):
    proc_json = [{
        'link': f"https://www.youtube.com/watch?v={item['id']['videoId']}",
        'title': item['snippet']['title'],
        'description': item['snippet']['description'],
        'thumb_img': item['snippet']['thumbnails']['high']['url'],
        'channel_owner': item['snippet']['channelTitle'],
        'publish_time': item['snippet']['publishTime'],
    } for item in json_data['items']]
    with open(f'gateway_outputs/youtube.json', 'w', encoding='utf-8') as f:
        json.dump(proc_json, f, ensure_ascii=False, indent=4)
    return proc_json


In [16]:
def parse_suggestion(device_json, youtube_json):
    sugg_json = {
        'n_devices': len(device_json.keys()),
        'perc_weekday': int(np.sum([0] + [1 for v in device_json.values() if v['perc_weekday'] > 80])),
        'mobile': int(np.sum([0] + [1 for v in device_json.values() if v['speed'] > 0.5])),
        'people': list(np.unique([x['channel_owner'] for x in youtube_json])),
    }
    with open(f'gateway_outputs/sugg.json', 'w', encoding='utf-8') as f:
        json.dump(sugg_json, f, ensure_ascii=False, indent=4)
    return sugg_json

In [17]:
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
geolocator = Nominatim(user_agent="example app")

In [18]:
app = Flask(__name__)
api = Api(app)
cors = CORS(app)


dataset_complete="gateway_outputs/list_start_end_locs.json"
dataset_trajectories="gateway_outputs/trajectories.json"
dataset_device = "gateway_outputs/device.json"
dataset_device_day = "gateway_outputs/device_day.json"

e="260BF5C2"


class location_map_static(Resource):

    def get(self):
        
        e=request.args.get('device')
        
        with open(dataset_trajectories) as json_file:
            data = json.load(json_file)
        
        path=[]
        for day in data[e]:
            try:
                latitude= data[e][day]["lat"][0]
                longitude= data[e][day]["lon"][0]
                error=200

                date=day

                latitude=float(latitude)
                longitude=float(longitude)

                m=generate_static_html_folium_marker(latitude,longitude,error,date)

                r=Response(m._repr_html_().encode('latin-1'))
                r.headers['Content-Type'] = 'text/html; charset=latin-1'

                return r
            except Exception as ex:
                print(ex)
                pass

class location_map_dynamic(Resource):
    
    def get(self):
        e=request.args.get('device')
        with open(dataset_trajectories) as json_file:
            data = json.load(json_file)

        path=[]
        for day in data[e]:
            try:
                lat_start= data[e][day]["lat"][0]
                lon_start= data[e][day]["lon"][0]

                lat_end= data[e][day]["lat"][len(data[e][day]["lat"])-1]
                lon_end= data[e][day]["lon"][len(data[e][day]["lon"])-1]

                date=day
                m=generate_dynamic_html_folium_marker(lat_start,lon_start,lat_end,lon_end,date)

                for i in range(len(data[e][day]["lat"])):    
                    location=(data[e][day]["lat"][i], data[e][day]["lon"][i])
                    path.append(location)

                folium.PolyLine(path,color='blue',weight=5,opacity=0.5).add_to(m)
                
                r=Response(m._repr_html_().encode('latin-1'))
                r.headers['Content-Type'] = 'text/html; charset=latin-1'

                return r            
            except Exception as ex:
                print(ex)
                pass
            
class get_weather(Resource):
    def get(self):
        e=request.args.get('device')
        
        with open(dataset_trajectories) as json_file:
            data = json.load(json_file)

        path=[]
        for day in data[e]:
            try:
                lat= data[e][day]["lat"][0]
                lon= data[e][day]["lon"][0]
                date=day

                resp = requests.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lon}/{date}?unitGroup=metric&key={api_key_weather}&include=current&contentType=json')
                data=resp.json()

                weather_res={}
                weather_res["address"]=data["address"]
                for d in data["days"]:
                    weather_res["tempmax"]=d["tempmax"]
                    weather_res["tempmin"]=d["tempmin"]
                    weather_res["precip"]=d["precip"]
                    weather_res["precipprob"]=d["precipprob"]
                    weather_res["humidity"]=d["humidity"]

                return weather_res
            except Exception as ex:
                print(ex)
                pass
            
class twitter_feed(Resource):
    def get(self):
        e=request.args.get('device')
        
        with open(dataset_trajectories) as json_file:
            data = json.load(json_file)

        path=[]
        for day in data[e]:
            try:
                lat= data[e][day]["lat"][0]
                lon= data[e][day]["lon"][0]
                date=day
                next_day = (datetime.strptime(date, '%Y-%m-%d').date() + relativedelta(days=1)).strftime('%Y-%m-%d')
                url="https://twitter.com/search?q=lora%20near%3A"+str(lat)+"%2C"+str(lon)+"%20until%3A"+str(next_day)+"%20since%3A"+str(date)+"&src=typed_query&f=live"
                return url
            except Exception as ex:
                print(ex)
                pass
            
class youtube_feed(Resource):
    def get(self):
        e=request.args.get('device')
        
        with open(dataset_trajectories) as json_file:
            data = json.load(json_file)

        path=[]
        for day in data[e]:
            try:
                lat= data[e][day]["lat"][0]
                lon= data[e][day]["lon"][0]
                date=day
                json_output=process_yb_json(parse_youtube(lat,lon,date))
                return json_output
            except Exception as ex:
                print(ex)
                pass
            
            
class facebook_feed(Resource):
    def get(self):
        e=request.args.get('device')
        
        with open(dataset_trajectories) as json_file:
            data = json.load(json_file)

        path=[]
        for day in data[e]:
            try:
                lat= data[e][day]["lat"][0]
                lon= data[e][day]["lon"][0]

                term=get_nearby_places(lat,lon,0)[0]
                term=str(urllib.parse.quote_plus(term))
                
                url= "https://www.facebook.com/events/search/?q="+term
                return url
            except Exception as ex:
                print(ex)
                pass
        
class raw_parser(Resource):
    def post(self):
        data = request.get_json()        
        device_json, dev_day_json, trajectory_json, list_json = mm.mobility(data)
        return list(device_json.keys())

    
class statistics(Resource):
    def get(self):
        e=request.args.get('device')
        with open(dataset_device) as json_file:
            data = json.load(json_file)
        return data[e]
            
class statistics_day(Resource):
    def get(self):
        e=request.args.get('device')
        with open(dataset_device_day) as json_file:
            data = json.load(json_file)
        return data[e]

class get_suggestions(Resource):
    def get(self):
        
        with open(dataset_device) as json_file:
            data = json.load(json_file)
        
        
        with open(dataset_trajectories) as json_file:
            data_youtube = json.load(json_file)

        path=[]
        try:
            for d in data_youtube.keys():
                for day in data_youtube[d]:
                    lat= data_youtube[d][day]["lat"][0]
                    lon= data_youtube[d][day]["lon"][0]
                    date=day
                    json_output=process_yb_json(parse_youtube(lat,lon,date))
                    
                    res=parse_suggestion(data,json_output)
                    
                    #res["Location info"]=get_cadastral_information(lat,lon)
                    
                    url_shodan,list_shodan=shodan_search(get_maps_province(lat,lon))
                    res["Shodan info"]=list_shodan
                                
                    return res
        
        except Exception as ex:
            print(ex)
            pass
        
    
    
    
def launch_REST_Server():
    api.add_resource(location_map_static, '/location_map_static')
    api.add_resource(location_map_dynamic, '/location_map_dynamic')
    api.add_resource(get_weather, '/get_weather')
    api.add_resource(twitter_feed, '/twitter_feed')
    api.add_resource(youtube_feed, '/youtube_feed')
    api.add_resource(facebook_feed, '/facebook_feed')
    api.add_resource(raw_parser,'/raw_parser')
    api.add_resource(statistics,'/statistics')
    api.add_resource(statistics_day,'/statistics_day')
    api.add_resource(get_suggestions,'/get_suggestions')

    
    http_server = WSGIServer(('0.0.0.0', 5002), app) #certfile="tls.crt",keyfile="tls.key
    http_server.serve_forever()


In [ ]:
launch_REST_Server()

list index out of range


127.0.0.1 - - [2022-10-06 17:42:12] "GET /twitter_feed?device=260B5F82 HTTP/1.1" 200 273 0.002991
<gevent._socket3.socket at 0x7f3c45ccf400 object, fd=60, family=2, type=1, proto=0>: Invalid HTTP method: '\x16\x03\x01\x02\x00\x01\x00\x01ü\x03\x03Á\x010\x1b\x19F\x86\'l\x00\x04\x10õâ\x9d)\x05\xa0\x1f|T¯\x0bL\xa0V~°8\x90pn \x1b\x1cLg\x80ÆLsßßãóèÊg\x90\x955TÐ$ÓÞ*Ê \x9eN\x86\x16\x86ô\x00"\x13\x01\x13\x03\x13\x02À+À/Ì©Ì¨À,À0À\n'
127.0.0.1 - - [2022-10-06 17:57:09] "  üÁ0F'l õâ) |T¯L V~°8pn LgÆLsßßãóèÊg5TÐ$ÓÞ*Ê Nô "À+À/Ì©Ì¨À,À0À" 400 - 0.001715
/tmp/ipykernel_2319144/3280268412.py:5: DeprecationWarning: ssl.SSLContext() without protocol argument is deprecated.
  gcontext = ssl.SSLContext()
/tmp/ipykernel_2319144/3280268412.py:5: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  gcontext = ssl.SSLContext()
127.0.0.1 - - [2022-10-06 17:57:19] "GET /location_map_dynamic?device=260BF5C2 HTTP/1.1" 200 409294 4.164357
192.168.18.174 - - [2022-10-06 18:34:08] "OP

192.168.18.174 - - [2022-10-06 18:34:12] "GET /youtube_feed?device=260BF5C2 HTTP/1.1" 200 698 0.616636
/tmp/ipykernel_2319144/3280268412.py:5: DeprecationWarning: ssl.SSLContext() without protocol argument is deprecated.
  gcontext = ssl.SSLContext()
/tmp/ipykernel_2319144/3280268412.py:5: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  gcontext = ssl.SSLContext()
192.168.18.174 - - [2022-10-06 18:34:22] "GET /get_suggestions HTTP/1.1" 200 1389 4.758295
192.168.18.174 - - [2022-10-06 18:34:22] "GET /location_map_dynamic?device=260BF5C2 HTTP/1.1" 200 409294 9.303658
192.168.18.174 - - [2022-10-06 18:34:22] "GET /twitter_feed?device=260BF5C2 HTTP/1.1" 200 307 0.001539
192.168.18.174 - - [2022-10-06 18:34:23] "GET /get_weather?device=260BF5C2 HTTP/1.1" 200 298 0.759917
